# FLASKED DEMO

This notebook is built towards showing a demonstration of the usage of the package.

NOTE: It is required the **requests** library to be able to contact the server and get responses in order to run this notebook.

In [1]:
!pip install requests

In [2]:
import sys
sys.path.insert(0, "..")

## IMPORTING THE PACKAGE
The package can be imported as follows

In [3]:
from flasked import Flasked

flasked = Flasked()
flasked

[Flasked flasked.core.flasked app: Not running; 0 routes defined]

## RUNNING THE SERVER

Server can be run in background taking advantage of the interactive environment of Jupyter notebooks or IPython:

In [4]:
flasked.run_background(host="127.0.0.1", port=1024)
flasked

[Flasked flasked.core.flasked app: Running on 127.0.0.1:1024; 0 routes defined]

 * Serving Flask app 'flasked.core.flasked'
 * Debug mode: off


 * Running on http://127.0.0.1:1024
Press CTRL+C to quit


Yes, I know there are no routes defined yet. However, the power of `Flasked` is that **routes can be added/modified/removed dynamically!**
Furthermore, the server can be stopped by using `flasked.stop()`. This is automatically handled by the framework, so re-creating the object and re-running the server on the same host-port configuration will automatically cancel previous servers:

In [5]:
flasked = Flasked()
flasked.run_background(host="127.0.0.1", port=1024)
flasked

[Flasked flasked.core.flasked app: Running on 127.0.0.1:1024; 0 routes defined]

 * Serving Flask app 'flasked.core.flasked'
 * Debug mode: off


 * Running on http://127.0.0.1:1024
Press CTRL+C to quit
127.0.0.1 - - [03/Mar/2024 23:11:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2024 23:11:40] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2024 23:11:40] "DELETE / HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2024 23:11:40] "PUT / HTTP/1.1" 405 -
127.0.0.1 - - [03/Mar/2024 23:11:40] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [03/Mar/2024 23:11:40] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2024 23:11:40] "DELETE / HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2024 23:11:40] "PUT / HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2024 23:11:40] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [03/Mar/2024 23:11:40] "POST / HTTP/1.1" 404 -
127.0.0.1 - - [03/Mar/2024 23:11:40] "DELETE / HTTP/1.1" 404 -
127.0.0.1 - - [03/Mar/2024 23:11:40] "PUT / HTTP/1.1" 404 -
127.0.0.1 - - [03/Mar/2024 23:11:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2024 23:11:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2024 23:11:40] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2024 23:

## SIMPLE ROUTES

In this section, generating a simple route and testing it will be shown, releasing the power of the framework.

### GENERATING A ROUTE

All the routes must be handled by a controller.

A controller is any class of Python containing as class methods the HTTP methods that are invoked during an HTTP call. For example, if you have a class that you want to make it a controller, you only need to define the get(), post() or equivalent HTTP method in the class. The `Flasked` framework will automatically instantiate your class on every request and forward the corresponding requests to this method of your class. Example:

In [6]:
class RootController:
    """The controller for root requests. It is a basic endpoint for demonstration of the package"""
        
    def get(self):
        return {"result": "my get result"}

    def post(self):
        return {"result": "my post result"}

    def delete(self):
        return {"result": "my delete result"}


We can assign this controller to any endpoint(s), but we will use the root "/" endpoint for convenience.

In [7]:
flasked['/'] = RootController

Now the `flasked` object handles this endpoint automatically. We can confirm this by showing a summary:

In [8]:
flasked.summary

[Flasked flasked.core.flasked app: Running on 127.0.0.1:1024; 1 routes defined]
	/ (GET, POST, DELETE) - RootController: The controller for root requests. It is a basic endpoint for demonstration of the package


Note that any arbitrary number of routes can be assigned to the flasked object, each pointing to the corresponding controller class.

### RUNNING THE ROUTE
The endpoint we just created is already available and can be used out-of-the-box:

In [9]:
%%time
import requests
display(requests.get("http://localhost:1024/").json())
display(requests.post("http://localhost:1024/").json())
display(requests.delete("http://localhost:1024/").json())
display(requests.put("http://localhost:1024/").json())

{'result': 'my get result'}

{'result': 'my post result'}

{'result': 'my delete result'}

{'error': 'Method not allowed'}

CPU times: user 48.1 ms, sys: 4.4 ms, total: 52.5 ms
Wall time: 68.6 ms


### MODIFYING ENDPOINTS

`Flasked` allows dynamic change of routes without needing to close or restart the server.

For example, we can redefine the class and assign it to the same route, replacing the existing one:

In [10]:
class RootController:
    """The controller for root requests. It is a basic endpoint for demonstration of the package"""
        
    def get(self):
        return {"result": "my get result, but returning a not found status code"}, 404

    def post(self):
        return {"result": "my post result"}

    def delete(self):
        return {"result": "my delete result"}

    def put(self):
        return {"result": "Now I have a put method handler"}

In [11]:
flasked['/'] = RootController

flasked.summary

[Flasked flasked.core.flasked app: Running on 127.0.0.1:1024; 1 routes defined]
	/ (GET, POST, PUT, DELETE) - RootController: The controller for root requests. It is a basic endpoint for demonstration of the package


In [12]:
%%time
import requests
display(requests.get("http://localhost:1024/").json())
display(requests.post("http://localhost:1024/").json())
display(requests.delete("http://localhost:1024/").json())
display(requests.put("http://localhost:1024/").json())

{'result': 'my get result, but returning a not found status code'}

{'result': 'my post result'}

{'result': 'my delete result'}

{'result': 'Now I have a put method handler'}

CPU times: user 13.2 ms, sys: 7.67 ms, total: 20.9 ms
Wall time: 18.7 ms


### REMOVING THE ROUTE
It is also possible to remove the route

In [13]:
del flasked['/']
flasked.summary

[Flasked flasked.core.flasked app: Running on 127.0.0.1:1024; 0 routes defined]
	


In [14]:
%%time
import requests
display(requests.get("http://localhost:1024/").json())
display(requests.post("http://localhost:1024/").json())
display(requests.delete("http://localhost:1024/").json())
display(requests.put("http://localhost:1024/").json())

{'error': 'Resource not found'}

{'error': 'Resource not found'}

{'error': 'Resource not found'}

{'error': 'Resource not found'}

CPU times: user 14.4 ms, sys: 4.81 ms, total: 19.2 ms
Wall time: 17.1 ms


### STATEFUL ROUTES

Sometimes, it is desired to keep a shared state between controller instances. For example, a database manager might be desired to be kept and accessable between instances of the same controller. This can be achieved with a singleton pattern or, alternatively, with dependency injection, which is the preferred way since it is allowed by `Flasked`.

In [15]:
class RootController:
    """The controller for root requests. It is a basic endpoint for demonstration of the package"""
    def __init__(self, stack_get, stack_post):
        self._stack_get = stack_get
        self._stack_post = stack_post
        
    def get(self):
        self._stack_get.append(1)
        return {"result": "my get result"}
        
    def post(self):
        self._stack_post.append(1)
        return {"result": "my post result"}

In [16]:
# Here it is shown how to keep a shared state by using two lists as example
my_get_stack = []
my_post_stack = []

flasked['/'] = RootController, {'stack_get': my_get_stack, 'stack_post': my_post_stack}

In [17]:
# Now we can perform the queries and state will be kept between requests:
display(requests.get("http://localhost:1024/").json())
display(requests.get("http://localhost:1024/").json())
display(requests.post("http://localhost:1024/").json())

{'result': 'my get result'}

{'result': 'my get result'}

{'result': 'my post result'}

In [18]:
display(my_get_stack)
display(my_post_stack)

[1, 1]

[1]

**Note: sharing state between instances face the issue of synchronizing data, since every request is potentially handled by different threads. Keep in mind that synchronization mechanisms might be desired. This example don't work when exposing in different WSGI scalable backends, which requires an external synchronization layer.**

## ADVANCE ROUTES

With `Flasked` it is also possible to handle special routes, like routes with variables in the path. This can be done in a similar manner as done with `Flask`, by using certain patterns in the endpoint definition (like \<variable>).

### UNTYPED VARIABLES
By default, variables not typped are treated as strings.

In [19]:
class AdvancedController:
    description = "The controller for advanced requests. It is an endpoint with variables in the path"
    
    def get(self, variable1, variable2):
        return {"result": "my get result", "variable1": variable1, "variable2": variable2}


flasked['/advanced/<variable1>/<variable2>'] = AdvancedController

**Note: Ensure that the name of the variables in the endpoint definition match the name of the variables in the class method parameter**

In [20]:
flasked.summary

[Flasked flasked.core.flasked app: Running on 127.0.0.1:1024; 2 routes defined]
	/advanced/<variable1>/<variable2> (GET) - AdvancedController: The controller for advanced requests. It is an endpoint with variables in the path
	/ (GET, POST) - RootController: The controller for root requests. It is a basic endpoint for demonstration of the package


In [21]:
requests.get("http://localhost:1024/advanced/foo/bar").json()

{'result': 'my get result', 'variable1': 'foo', 'variable2': 'bar'}

### TYPED VARIABLES

Variables can be typped. Allowed types are `int`, `float` and `string`:

In [22]:
class AdvancedControllerTyped:
    description = "The controller for advanced requests. It is an endpoint with variables in the path"
    
    def get(self, variable1, variable2, variable3):
        return {"result": "my get result", "variable1": variable1, "variable2": variable2, "variable3": variable3}


flasked['/advanced/<int:variable1>/<float:variable2>/<string:variable3>'] = AdvancedControllerTyped

In [23]:
flasked.summary

[Flasked flasked.core.flasked app: Running on 127.0.0.1:1024; 3 routes defined]
	/advanced/<int:variable1>/<float:variable2>/<string:variable3> (GET) - AdvancedControllerTyped: The controller for advanced requests. It is an endpoint with variables in the path
	/advanced/<variable1>/<variable2> (GET) - AdvancedController: The controller for advanced requests. It is an endpoint with variables in the path
	/ (GET, POST) - RootController: The controller for root requests. It is a basic endpoint for demonstration of the package


In [24]:
requests.get("http://localhost:1024/advanced/24/1.1/foo").json()

{'result': 'my get result',
 'variable1': 24,
 'variable2': 1.1,
 'variable3': 'foo'}

If a variable type is not correctly detected (for example, passing a string to a float or int variable), it will not be recognized:

In [25]:
requests.get("http://localhost:1024/advanced/bar/1.1/foo").json()

{'error': 'Resource not found'}

### PATH VARIABLES

Path variables can be used to fetch dynamic paths in the endpoints, in a similar manner as Flask allows:

In [26]:
class AdvancedControllerPath:
    description = "The controller for requests with path variables"
    
    def get(self, path_variable):
        return {"result": "my get result", "path_variable": path_variable}


flasked['/advanced/path/<path:path_variable>/'] = AdvancedControllerPath

In [27]:
flasked.summary

[Flasked flasked.core.flasked app: Running on 127.0.0.1:1024; 4 routes defined]
	/advanced/<int:variable1>/<float:variable2>/<string:variable3> (GET) - AdvancedControllerTyped: The controller for advanced requests. It is an endpoint with variables in the path
	/advanced/path/<path:path_variable>/ (GET) - AdvancedControllerPath: The controller for requests with path variables
	/advanced/<variable1>/<variable2> (GET) - AdvancedController: The controller for advanced requests. It is an endpoint with variables in the path
	/ (GET, POST) - RootController: The controller for root requests. It is a basic endpoint for demonstration of the package


In [28]:
requests.get("http://localhost:1024/advanced/path/bar/1.1/foo/").json()

{'path_variable': 'bar/1.1/foo', 'result': 'my get result'}

It can also work with many path variables or mixing variables:

In [29]:
class AdvancedControllerPathComplex:
    description = "The controller for requests with path variables"
    
    def get(self, path_variable1, path_variable2):
        return {"result": "my get result", "path_variable1": path_variable1, "path_variable2": path_variable2}


flasked['/advanced/path/<path:path_variable1>/path2/<path:path_variable2>'] = AdvancedControllerPathComplex

In [30]:
flasked.summary

[Flasked flasked.core.flasked app: Running on 127.0.0.1:1024; 5 routes defined]
	/advanced/path/<path:path_variable1>/path2/<path:path_variable2> (GET) - AdvancedControllerPathComplex: The controller for requests with path variables
	/advanced/<int:variable1>/<float:variable2>/<string:variable3> (GET) - AdvancedControllerTyped: The controller for advanced requests. It is an endpoint with variables in the path
	/advanced/path/<path:path_variable>/ (GET) - AdvancedControllerPath: The controller for requests with path variables
	/advanced/<variable1>/<variable2> (GET) - AdvancedController: The controller for advanced requests. It is an endpoint with variables in the path
	/ (GET, POST) - RootController: The controller for root requests. It is a basic endpoint for demonstration of the package


In [31]:
requests.get("http://localhost:1024/advanced/path/bar/1.1/foo/path2/foo/bar").json()

{'path_variable1': 'bar/1.1/foo',
 'path_variable2': 'foo/bar',
 'result': 'my get result'}

### PARSING URL PARAMETERS

Routes can have parameters. They can be accessed in the same way done with Flask. Remember this framework is flask-based:

In [32]:
from flask import request


class ArgumentsParserController:
    def get(self):
        parameter_value = request.args.get('parameter')
        return {"parameter": parameter_value}

flasked["/myparser"] = ArgumentsParserController

In [33]:
flasked.summary

[Flasked flasked.core.flasked app: Running on 127.0.0.1:1024; 6 routes defined]
	/advanced/path/<path:path_variable1>/path2/<path:path_variable2> (GET) - AdvancedControllerPathComplex: The controller for requests with path variables
	/advanced/<int:variable1>/<float:variable2>/<string:variable3> (GET) - AdvancedControllerTyped: The controller for advanced requests. It is an endpoint with variables in the path
	/advanced/path/<path:path_variable>/ (GET) - AdvancedControllerPath: The controller for requests with path variables
	/advanced/<variable1>/<variable2> (GET) - AdvancedController: The controller for advanced requests. It is an endpoint with variables in the path
	/myparser (GET) - ArgumentsParserController: None
	/ (GET, POST) - RootController: The controller for root requests. It is a basic endpoint for demonstration of the package


In [34]:
requests.get("http://localhost:1024/myparser?parameter=24").json()

{'parameter': '24'}

### PARSING JSON REQUESTS

In the same way as usually done with Flask:

In [35]:
from flask import request


class JSONParserController:
    
    def post(self):
        json_content = request.get_json()
        return {"json": json_content}

flasked["/myjson"] = JSONParserController

In [36]:
flasked.summary

[Flasked flasked.core.flasked app: Running on 127.0.0.1:1024; 7 routes defined]
	/advanced/path/<path:path_variable1>/path2/<path:path_variable2> (GET) - AdvancedControllerPathComplex: The controller for requests with path variables
	/advanced/<int:variable1>/<float:variable2>/<string:variable3> (GET) - AdvancedControllerTyped: The controller for advanced requests. It is an endpoint with variables in the path
	/advanced/path/<path:path_variable>/ (GET) - AdvancedControllerPath: The controller for requests with path variables
	/advanced/<variable1>/<variable2> (GET) - AdvancedController: The controller for advanced requests. It is an endpoint with variables in the path
	/myparser (GET) - ArgumentsParserController: None
	/myjson (POST) - JSONParserController: None
	/ (GET, POST) - RootController: The controller for root requests. It is a basic endpoint for demonstration of the package


In [37]:
import json

requests.post("http://localhost:1024/myjson", 
              data=json.dumps({"foo": "bar"}), 
              headers={'content-type': 'application/json'}).json()

{'json': {'foo': 'bar'}}